In [5]:
import os
import datetime
import time
from dotenv import load_dotenv
import anthropic
from firecrawl import FirecrawlApp
import json
import requests
import os
import json

# Load environment variables
load_dotenv()

# Retrieve API keys from environment variables
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
firecrawl_api_key = os.getenv("FIRECRAWL_API_KEY")


In [7]:
# Initialize the FirecrawlApp with your API key
app = FirecrawlApp(api_key=firecrawl_api_key)

# Crawl a website
crawl_url = 'https://dify.ai/'
params = {
    'crawlOptions': {
        'limit': 10
    }
}
crawl_result = app.crawl_url(crawl_url, params=params)

if crawl_result is not None:
    # Convert crawl results to JSON format, excluding 'content' field from each entry
    cleaned_crawl_result = [{k: v for k, v in entry.items() if k != 'content'} for entry in crawl_result]

    # Save the modified results as a text file containing JSON data
    with open('crawl_result.txt', 'w') as file:
        file.write(json.dumps(cleaned_crawl_result, indent=4))
else:
    print("No data returned from crawl.")


In [21]:


website_dump = open('crawl_result.txt', 'r').read()

anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

headers = {
    "content-type": "application/json",
    "x-api-key": anthropic_api_key,
    "anthropic-version": "2023-06-01",
    "anthropic-beta": "prompt-caching-2024-07-31"
}

data = {
    "model": "claude-3-5-sonnet-20240620",
    "max_tokens": 1024,
    "system": [
        {
            "type": "text",
            "text": "You are an AI assistant tasked with analyzing websites for copy errors.\n"
        },
        {
            "type": "text",
            "text": website_dump,
            "cache_control": {"type": "ephemeral"}
        }
    ],
    "messages": [
        {
            "role": "user",
            "content": "How can I improve the copy on this website?"
        }
    ]
}

response = requests.post(
    "https://api.anthropic.com/v1/messages",
    headers=headers,
    data=json.dumps(data)
)

print(response.json())



{'id': 'msg_01A529h2zHKLCxKAgTiNc4P1', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-5-sonnet-20240620', 'content': [{'type': 'text', 'text': 'Based on my analysis of the website content, here are some suggestions to improve the copy:\n\n1. Streamline the messaging: The homepage contains a lot of information, which can be overwhelming. Consider focusing on 3-5 key benefits or features and elaborate on those, rather than trying to cover everything.\n\n2. Clarify the value proposition: While the tagline "The Innovation Engine for GenAI Applications" is catchy, it may not immediately convey what Dify does. Consider a more specific tagline that explains the core benefit to users.\n\n3. Use more concrete language: Some descriptions are quite abstract (e.g., "Fortify apps securely with reliable data pipelines"). Try to use more specific, tangible language to describe features and benefits.\n\n4. Improve consistency: The tone and style of writing varies throughout the page. Aim f